In [ ]:
from glob import glob
import seba

In [ ]:
data_folder = r"C:\Ephys_analysis\data\Ephys\NP2_test" #folder that contains folders with ephys recordings
histology_folder = r"C:\Ephys_analysis\data\NP_tiled" #folder that contains folders with HERBS output after drawing probes onto the images
filepaths = glob(r"C:\Ephys_analysis\data\behav_annot_old\*.bvs") #path to behaview output of behaview behavior annotation
framert = [30, 50, 50, 50, 50, 50] #list of framerates of videos
save_path = glob(r"C:\Ephys_analysis\data\Ephys\*") #list of folders with ephys_recordings
save_results=r"C:\Ephys_analysis\data\Results_test" #folder in which the data structure and plots will be saved

In [ ]:
#If using BehaView
for file, fps, save in zip(filepaths, framert, save_path): 
     seba.data.io.read_bvs(filepath=file, framerate=fps, save_path=save)
seba.extract_raw_events_TS_BehaView(data_folder=[data_folder], framerate=30, append_CS=True, CS_filename="TTL_3.txt", CS_len=20)

In [ ]:
#If using boris annotations - assumes that for each recording annotations from boris were manually moved to corresponding folder
seba.read_extract_boris(data_folder=[data_folder], boris_output="boris_output.csv", 
                      camera_TTL="cam_TTL.txt", append_CS=True, CS_filename="TTL_3.txt", CS_len=20, padding=False)              

In [ ]:
#Get histology data
#If using neuropixels 2.0. If not, skip first function
seba.fix_wrong_shank_NP2([data_folder])
seba.get_brain_regions([data_folder], histology_folder, neuropixels_20=True)

In [ ]:
#Build data structure
data_obj = seba.structurize_data([data_folder], save_path=save_results, pre_event=2, post_event=4, bin_size=0.01, calculate_responsive=True, p_bound=0.05)
#Save information about responsivity to cluster_info_good.csv
seba.responsive_neurons2events([data_folder], data_obj)

In [ ]:
#Define pairs of behaviors you want to explore
behavioral_pairs = [["freezing_ephys_onsets", "freezing_non_ephys_onsets"],
                    ["rearing_ephys_onsets", "rearing_non_ephys_onsets"],
                    ["freezing_ephys_onsets", "CS_onsets"],
                    ["rearing_ephys_onsets", "social_onsets"],
                    ["freezing_ephys_offsets", "freezing_non_ephys_offsets"],
                    ["rearing_ephys_offsets", "rearing_non_ephys_offsets"],
                    ["freezing_ephys_offsets", "CS_offsets"],
                    ["rearing_ephys_offsets", "social_offsets"],
                    ["freezing_ephys_onsets", "freezing_ephys_offsets"],
                    ["rearing_ephys_onsets", "rearing_ephys_offsets"],
                    ["CS_onsets", "CS_offsets"],
                    ["social_onsets", "social_offsets"],
                    ]

In [ ]:
seba.plot_common_nrns_matrix(data_obj, save_results, per_animal=False, percent_all_neurons=False)
#Plots everything
for pair in behavioral_pairs:
    seba.plot_heatmaps_paired(data_obj, pair, save_path=save_results, per_animal=False, responsive_only=True, colormap = "inferno", z_score=True)
    seba.plot_lin_reg_scatter(data_obj, pair, save_path=save_results, per_animal=False, responsive_only=True)
    seba.plot_psths_paired(data_obj, pair, save_path=save_results, responsive_only=True, z_score=True, ylimit= [-1, 3])

seba.plot_psths(data_obj, save_results, ylimit= [-1, 3], responsive_only=True)
seba.plot_heatmaps(data_obj, save_results, responsive_only=True)
seba.neurons_per_structure([data_folder], data_obj, save_results)
seba.neurons_per_event([data_folder], data_obj, save_results)